In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: 
# encode("gooz")
# for i in encode("gooz"):
#     print(itos[i])

print(encode("hii there!"))
print(decode(encode("hii there!")))

[46, 47, 47, 1, 58, 46, 43, 56, 43, 2]
hii there!


In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # the 1000 characters we looked at earlier will look like this to GPT

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
n = int(len(data)* 0.9)
train_data = data[:n]
val_data = data[n:]
print(len(train_data))
print(len(val_data))

1003854
111540


In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

# generate a small batch of data of inputs x and targets y
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b,:t+1]
        target = yb[b,t]
        # print(f"when input is {context.tolist()} the target is: {target} ")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [56]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

# hyperparameters
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 4
n_layer = 3
dropout = 0.2
# ---------------------

class Head(nn.Module):
	""" one head of self attention """

	def __init__(self, head_size):
		super().__init__()
		self.key = nn.Linear(n_embd, head_size, bias=False)
		self.query = nn.Linear(n_embd, head_size, bias=False)
		self.value = nn.Linear(n_embd, head_size, bias=False)
		self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # tril is not a parameter of the module, so it's called buffer in pytorch naming convention it's called buffer 
		self.dropout = nn.Dropout(dropout) # randomly prevent some nodes from communicating

	def forward(self, x):
		B,T,C = x.shape
		k = self.query(x) # (B, T, C)
		q = self.query(x) # (B, T, C)
		# compute attention scores ("affinities")
		wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, T)
		wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
		wei = F.softmax(wei, dim=-1) # (B, T, T)
		wei = self.dropout(wei)
		# perform the weighted aggregation of the values
		v = self.value(x) # (B, T, C)
		out = wei @ v # (B, T, C)
		return out

class MultiHeadAttention(nn.Module):
	""" multiple heads of self-attention in parallel"""

	def __init__(self, num_heads, head_size):
		super().__init__()
		self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
		self.proj = nn.Linear(n_embd, n_embd) # Project the residual connections back into a linear layer, mixing the two forked paths
		self.dropout = nn.Dropout(dropout)

	def forward(self, x):
		out = torch.cat([h(x) for h in self.heads], dim=-1)
		out = self.dropout(self.proj(out))
		return out

class FeedFoward(nn.Module):
	""" a simple linear layer followed by a non-linearity """

	def __init__(self, n_embd):
		super().__init__()
		self.net = nn.Sequential(
				nn.Linear(n_embd, 4 * n_embd), # We multiple inner layer of feedforward from the Transformers paper. It adds computation and growing layer in the residual block 
				nn.ReLU(),
				nn.Linear(4 * n_embd, n_embd), # Projection layer going back into residual pathway
				nn.Dropout(dropout)
			)

	def forward(self, x):
		return self.net(x)


class Block(nn.Module):
	""" Transformer block: communication followed by computation """

	def __init__(self, n_embd, n_head):
		super().__init__()
		head_size = n_embd // n_head
		self.sa_heads = MultiHeadAttention(n_head, head_size) # i.e. 4 heads of 8-dimensional self-attention
		self.ffwd = FeedFoward(n_embd)
		self.ln1 = nn.LayerNorm(n_embd)
		self.ln2 = nn.LayerNorm(n_embd)

	def forward(self, x):
		x = x + self.sa_heads(self.ln1(x)) # apply one head of self-attention. (B, T, C)
		x = x + self.ffwd(self.ln2(x)) # (B, T, C)
		return x


class BigramLanguageModel(nn.Module):

	def __init__(self):
		super().__init__()
		self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
		self.position_embedding_table = nn.Embedding(block_size, n_embd)
		self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) # The asterick unpacks the list since pytorch wants comma separated
		# self.blocks = nn.Sequential(
		# 	Block(n_embd, n_head=4),
		# 	Block(n_embd, n_head=4),
		# 	Block(n_embd, n_head=4),
		# 	nn.LayerNorm(n_embd),
		# )
		self.ln_f = nn.LayerNorm(n_embd) # final layer norm
		self.lm_head = nn.Linear(n_embd, vocab_size)
		
	def forward(self, idx, targets=None):
		B, T = idx.shape

		# idx and targets are both (B, T) tensor of integers
		tok_emb = self.token_embedding_table(idx) # (B, T, C)
		pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
		x = tok_emb + pos_emb # (B, T, C)
		x = self.blocks(x) # (B, T, C)
		x = self.ln_f(x)
		logits = self.lm_head(x) # (B, T, vocab_size)

		# Note that if targets are None, to view operation is done, it's just the embedding above with B, T, C shape
		if targets is None:
			loss = None
		else:
			# reshape for pytorch
			B, T, C = logits.shape
			logits = logits.view(B*T, C) # We do this beause pytorch requires specific shape for cross entropy, so we convert to 2 dimensional. B*T, C stretches out the array while preserving the C values
			targets = targets.view(B*T)
			loss = F.cross_entropy(logits, targets)

		return logits, loss

	def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
		for _ in range(max_new_tokens):
			# crop idx to the last block_size tokens
			idx_cond = idx[:, -block_size:]
			# Forward the model / get predictions
			logits, loss = self(idx_cond) # (B, T, C)
			# focus only on the last time step
			logits = logits[:, -1, :] # (B, C) where it only grabs the last token in the sequence since this can only be used for next char probs
			# apply softmax to get probs
			probs = F.softmax(logits, dim=-1) # (B, C) -  Since it's a 1x65, give the probabilities along those 65
			# sample from distribution
			idx_next = torch.multinomial(probs, num_samples=1)
			# append sampled index to the running sequence
			idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
		return idx
        
m = BigramLanguageModel()
logits, loss = m(xb, yb)
print(logits.shape, loss)

torch.Size([32, 65]) tensor(4.2061, grad_fn=<NllLossBackward0>)


In [67]:
# class MultiHeadAttention(nn.Module):
# 	""" multiple heads of self-attention in parallel"""

# 	def __init__(self, num_heads, head_size):
# 		super().__init__()
# 		self.heads = [Head(head_size) for _ in num_heads]

heads = [Head(8) for _ in range(4)]

# # test = torch.stack([xb, xb], dim=0)
test = m.token_embedding_table(xb)
test.shape

# heads[0](test)

# # for h in heads:
# #     print(h(test))

total = [h(test) for h in heads]
total[1].shape
torch .cat(total, dim=-1).shape


torch.Size([4, 8, 32])

In [ ]:
logits, loss = m(xb[:4])
# xb[:4]

xb[0], logits[0]

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100):

    # sample the batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss)

In [ ]:
decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=200)[0].tolist())

In [ ]:
for p in m.parameters():
    print(p.data) 

# m.parameters()

In [ ]:
m.token_embedding_table.weight[0]
m(torch.tensor([0]))[0]

In [ ]:
torch.manual_seed(42)
# a = torch.ones(3, 3)
a = torch.tril(torch.ones(3, 3), diagonal=0)# / a.sum(1)
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
# c = (a @ b) / a.sum(1).view(3, -1)

print(a)
print(b)
print(c)

# a @ b = c
# c.shape, a.sum(1).shape, a.sum(1).view(3, -1)

# c / a.sum(1).view(3, -1)

In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape
x

# x, x.view(B*T, C)

In [ ]:
# we want x[b,t] = mean_{i<=t} x[b,i]

# version 1
xbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (T,C)
        xbow[b,t] = torch.mean(xprev, 0)

x[0], xbow[0], torch.mean(x[0], 0)

In [ ]:
# version 2
# 4x8x8 @ 4x8x2 --> 4x8x2

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
# xbow2[0] == xbow[0]
# torch.allclose(xbow, xbow2)

In [ ]:
# version 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
# torch.allclose(xbow3, xbow)

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels/classes
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
out.shape

# k.transpose(-1).shape

# v = value(x) 

# q.shape, k.shape
# q.shape, k.transpose(-2, -1).shape
# q[0].shape, k.transpose(-2, -1)[0].shape
# q[0] @ k.transpose(-2, -1)[0]
# # q[0]
# F.softmax((q[0] @ k.transpose(-2, -1)[0]), dim=-1)

# k.var(), q.var(), wei.var()